In [181]:
data="""
[F8] [F9] [F10]

[F102]

[F103] [F104] [F105] [F106]-[F107]

Dear [FS] [F10]

We're writing to tell you that your Part D late enrollment penalty has changed based on Medicare's reconsideration (review) of your circumstances. This was explained in the decision letter you got from Medicare's Appeals Contractor dated [F142].

[(if F36=851 and F66> $0.00 and F135>$0.00 and F130≠C, populate) As a result of Medicare's reconsideration decision, your premium still includes a late enrollment penalty. Of Account≠8013OPT46S, populate Please contact your former employer's group/union for your premium information

Since you owe a late enrollment penalty dating back to your effective date of enrollment, we'll charge you a lump sum amount of [F135]]
[(if F36=851 and F66> $0.00 and F135>$0.00 and F130≠C, populate)]

[(If F36=852 and F66>$0.00 and F135>$0.00 and F230≠C, populate) As a result of Medicare's reconsideration decision you still owe a penalty of E661 per month,

effective [F131] [Of Account 80130PT165. populatelPlease contact your former]

employer's group/union for your premium information Since you owe a late enrollment penalty dating back to your effective date of enrollment, we'll charge you a lump sum amount of [135]]

[(If F36=851 or 852 and F66=0.00 and F130=C and F134 = $0.00 populate) As a

result of Medicare's reconsideration decision], your premium still includes a late enrollment penalty Kif Accounte80130PT-165, potulate Please contact your former employer's group/union for your premium information

Since you owe a late enrollment penalty dating back to your effective date of enrollment, we'll charge you a lump sum amount of [13511

[Account 80130PT165, pupulatelyour former employer's group/union has agreed to pay [F134]], the amount of your late enrollment penalty, on your behalf. If your coverage is terminated by you or your former employer's group/union, or
]"""

In [198]:
import docx2txt
import re

# FILE_PATH=input("Enter template path(File should be of .docx format) ->   ")
FILE_PATH='doc1.docx'
try:
#     file_data = docx2txt.process(FILE_PATH)
    file_data=data
except:
    print("Oh No ! ..Error file name is missing in the path ("+FILE_PATH+") mentioned or error with file format")
    
def error_index_conditions(error_index,test):
    forward_index=20
    ERROR_MESSAGE='Oops ! Bracket seems to have an issue between these characters -> '
    if error_index+forward_index>len(test) and error_index-forward_index>0:
        return ERROR_MESSAGE+test[error_index-forward_index:error_index]
    elif error_index-forward_index<0 and error_index+forward_index<len(test):
        return ERROR_MESSAGE+test[error_index:error_index+forward_index]
    else:
        return ERROR_MESSAGE+test[error_index-forward_index:error_index+forward_index]
    
def check_balanced_brackets(test):
    print("Checking Bracket Count!!")
    stack=[]
    open_list = ["[","("]
    close_list = ["]",")"]
    
    pos=0
    for i in range(0,len(test)):
        
        if test[i] in open_list:
            stack.append([test[i],i])
        elif test[i] in close_list:
        
            pos=close_list.index(test[i])

            if ((len(stack) > 0) and (open_list[pos] == stack[len(stack)-1][0])):
                stack.pop()
            else:
                
                if len(stack)==0:
                    error_index=i
                else:
                    error_index=stack[0][1]
                return error_index_conditions(error_index,test)
    
    if len(stack) == 0:
        return "Brackets count seems to be ok!!"
    else:
        error_index=stack[0][1]
        return error_index_conditions(error_index,test)
    
#Ist rule to check if brackets count is same or if they are balanced or not
data_remove_new_line=re.sub('\n'," ",file_data)
try:
    print(check_balanced_brackets(data_remove_new_line))
except:
    print("ERROR! checking count of brackets ! Please look in the template for BRACKETS")

print("Checking for logic part!!")
# 2nd RULE : Check logic part of template

data_new_line_split=list(filter(None,data.split("\n")))

#Assume 1: There is if and populate in all the data lines
count_expression=0
for data_item in data_new_line_split:
    logic_parts=re.findall('\[\((if)(.*?)(populate\))',data_item,re.IGNORECASE)
    
    for logic_part in logic_parts:
        
        count_expression+=1
# checking logic part 0
        if logic_part[0][0].islower():
            print("Oops ! 'i' in if  is not UpperCase for Logic -> "+"".join(logic_part))
            
# checking logic part 1
        bracket_keys=re.search('\[[A-Z]{1}[a-z]+[A-Z]{1}[a-z]+\]|\[F\d+\]',logic_part[1])

        if bracket_keys:
            print("Logic part has keys with brackets -> "+logic_part[1])
        if logic_part[1][0]!=' ':
            print("Oops! No space between If and expression ->"+"".join(logic_part))
        if re.search('\s{2,}',logic_part[1]):
            print("Oops! More than 2  spaces between If and expression ->"+"".join(logic_part))
        if re.search('logic|of|AND|OR|IS NOT|is not|&|:',logic_part[1]):
            print("Oops! Issue with this part of expression(It contains these Characters - logic|of|AND|OR|IS NOT|is not|&|:) ->"+"".join(logic_part))
        
        if logic_part[1][-1]!=' ' or logic_part[1][-2]!=',':
            print("Oops! No space or comma with this part of expression ->"+"".join(logic_part))
        if re.search(",",logic_part[1][1:-2]):
            print("Oops! comma found with this part of expression ->"+"".join(logic_part))
    

#Assume 2: There is no if in data lines

count_populate=data_remove_new_line.count("populate)")
count_If=data_remove_new_line.count("(If")
if count_If!=count_populate:
    missing_If_or_populate_count=abs(count_populate-count_If)
    print(str(missing_If_or_populate_count)+" 'If' or 'populate' seems to be missing in some of populate expression")
elif count_If!=count_expression:
    print("There is a issue with the expressions in template Seems like 'If' doesn't start with [(")

#Assume 3: There is word logic in data lines

for data_item in data_new_line_split:
    logic_parts=re.findall('\((logic)(.*?populate\))',data_item,re.IGNORECASE)
    if len(logic_parts)!=0:
        print("Word 'logic' found in this part of expression ->","".join(logic_parts[0]))
        

# 3rd RULE : Right syntax for keys

search_data_for_keys=re.sub('[if|logic].*?populate',"",data_remove_new_line,flags=re.IGNORECASE)
keys_exp_without_brack=re.findall('[A-Z]{1}[a-z]+[A-Z]{1}[a-z]+|F\d+',search_data_for_keys)
# keys_exp_without_brack=["[" + s +"]" for s in keys_exp_without_brack]
keys_exp_with_brack=re.findall('\[[A-Z]{1}[a-z]+[A-Z]{1}[a-z]+\]|\[F\d+\]',search_data_for_keys)
keys_error_values=[i for i, j in zip(keys_exp_without_brack, keys_exp_with_brack)  if "["+i+"]" != j ] 
if len(keys_error_values)!=0:
    print("Issue with keys not in brackets ->",keys_error_values)



# 4th RULE : Address syntax

for data_item in data_new_line_split:
    address_part=re.findall('\[[A-Z]+[a-z]+[A-Z]+\w+\]',data_item)
    if len(address_part)==4 and not re.search('\[[A-Z]+[a-z]+[A-Z]+\w+\], \[[A-Z]+[a-z]+[A-Z]+\w+\], \[[A-Z]+[a-z]+[A-Z]+\w+\] \[[A-Z]+[a-z]+[A-Z]+\w+\]',data_item.strip()):
        print("address format seems to be incorrect for string -> "+data_item)
        
    
# 5th RULE : dash check for keys

for data_item in data_new_line_split:

    dash_check=re.search('\[F\d+\]-\[F\d+\]',data_item)
    if dash_check:
        string_r=re.sub('\[|\]','',dash_check.group())
        print("replace dash with ",'['+string_r.split('-')[0]+']'+'[(If '+string_r.split('-')[1]+'≠BLANK, populate)-]['+string_r.split('-')[1]+']')


Checking Bracket Count!!
Brackets count seems to be ok!!
Checking for logic part!!
Oops ! 'i' in if  is not UpperCase for Logic -> if F36=851 and F66> $0.00 and F135>$0.00 and F130≠C, populate)
Oops ! 'i' in if  is not UpperCase for Logic -> if F36=851 and F66> $0.00 and F135>$0.00 and F130≠C, populate)
Oops! No space or comma with this part of expression ->If F36=851 or 852 and F66=0.00 and F130=C and F134 = $0.00 populate)
2 'If' or 'populate' seems to be missing in some of populate expression
replace dash with  [F106][(If F107≠BLANK, populate)-][F107]


In [194]:
string="""
[(If F36=852 and F66>$0.00 and F135>$0.00 and F230≠C, populate) As a res
"""
s=re.sub('if.*?populate',"",string,flags=re.IGNORECASE)
print(s)


[() As a res



In [86]:
logic_part[1][1:-2]

'F36=852 and F66>$0.00 and F135>$0.00 and F230≠C'

In [176]:
a=[2,3,4]
b=[4,5,1]
set(a)-set(b)

{2, 3}